In [3]:
# Вычислить значение интеграла функции -3.6 * (x**4) + 6.1 * (x**3) - 4.4  * (x**2) + 6.5 * x + 0.8 в пределах от 1 до 3.
# Аналитически полученное значение ~ -62.7733333

import numpy as np
import matplotlib as plot
import math
import scipy.integrate as integr

# Отрезок и коэффициенты многочлена от c_0 до c_4
a = 1
b = 3
polynomial = np.array([0.8, 6.5, -4.4, 6.1, -3.6])

def H_Left(M1, eps, a, b):
    h_left =  2 * eps / (M1 * (b - a))
    n_left = int((b - a) / h_left) + 1
    h_left = (b - a)/n_left
    #h_left = (b-a)
    #n_left = 1
    #while(h_left >= 2 * eps / (M1 * (b-a))):
    #    h_left /= 2
    #    n_left *= 2
    
    return h_left, n_left

def H_S(M4, eps, a, b):
    h =  (2880 * eps / (M4 * (b - a)))**(1/4)
    n = int((b - a) / h) + 1
    h = (b - a)/n
    #h = (b-a)
    #n = 1
    #while(h >= (2880 * eps / (M4 * (b-a)))**(1/4)):
    #    h /= 2
    #    n *= 2
    return h, n

# Расчёт значения полинома в точке
def FunctionPolynomial(x):
    global polynomial
    result = 0
    for i in range(len(polynomial)):
        result += polynomial[i] * x**i
    return result

# Расчёт по формуле левых прямоугольников для разбиения на n+1 точку с длиной отрезков h 
def LeftRiemannSum(f, h, n, a, b):
    point = a
    result = 0
    for i in range(n):
        result += f(point)
        point += h
    result *= h
    return result

# Расчёт по формуле левых прямоугольников значения интеграла и разбиения, удовлетворяющих точности
# (использует прошлую функцию для значения для конкретного разбиения)
def LeftRiemannSumByEps(f, a, b, eps):
    n_left = 1
    h_left = (b - a)
    I_LR_prev = LeftRiemannSum(f, h_left, n_left, a, b)
    I_LR = LeftRiemannSum(f, h_left/2, n_left*2, a, b)
    while(Runge(I_LR, I_LR_prev, 1) > eps):
        h_left /= 2
        n_left *= 2
        I_LR_prev = I_LR
        I_LR = LeftRiemannSum(f, h_left/2, n_left*2, a, b)
    h_left /= 2
    n_left *= 2
    return I_LR, I_LR_prev, h_left, n_left

# Расчёт по формуле Симпсона для конкретного разбиения
def SimpsonRule(f, h, n, a, b):
    result = f(a) + f(b) + 4 * f(a + h/2.0)
    point = a + h
    for i in range(1, n):
        result += 4 * f(point + h/2.0) + 2 * f(point)
        point += h
    result *= h/6.0
    return result

# Проверка погрешности по правилу Рунге
def Runge(I_h, I_2h, p):
    return abs((I_h - I_2h)/(2**p - 1))

# Расчёт по формуле Симпсона значения интеграла и разбиения, удовлетворяющих точности
def SimpsonRuleByEps(f, a, b, eps):
    n_S = 1
    h_S = (b - a)
    #n_S = 40
    #h_S = (b - a)/n_S
    I_S_prev = SimpsonRule(f, h_S, n_S, a, b)
    I_S = SimpsonRule(f, h_S/2, n_S*2, a, b)
    while(Runge(I_S, I_S_prev, 4) > eps):
        h_S /= 2
        n_S *= 2
        I_S_prev = I_S
        I_S = SimpsonRule(f, h_S/2, n_S*2, a, b)
    h_S /= 2
    n_S *= 2
    return I_S, I_S_prev, h_S, n_S

# Расчёт по формуле Гаусса с 3 узлами
def Gauss_3(f):
    return 5/9 * f(2 - math.sqrt(3/5)) + 8/9 * f(2) + 5/9 * f(2 + math.sqrt(3/5))

I = -62.77333333333333333333

# Точности расчёта для формул левых прямоугольников и Симпсона
eps_left = 1e-3
eps_S = 1e-3

# Максимальные значения модуля производной 1-го и 4-го порядка соответсвенно
M1 = 244
M4 = 86.6

h_left, n_left = H_Left(M1, eps_left, a, b)
h_S, n_S = H_S(M4, eps_S, a, b)

h_left, n_left = H_Left(M1, eps_left, a, b)
print('Шаг интегрирования для формулы левых прямоугольников:', h_left)
print('Число разбиений для формулы левых прямоугольников:', n_left)
h_S, n_S = H_S(M4, eps_S, a, b)
print('Шаг интегрирования для формулы Симпсона:', h_S)
print('Число разбиений для формулы Симпсона:', n_S)

I_left = LeftRiemannSum(FunctionPolynomial, h_left, n_left, a, b)
I_S = SimpsonRule(FunctionPolynomial, h_S, n_S, a, b)

print('I =', I)
print('eps =', eps_left, ', I_left =', I_left, ', R = |I - I_left| =', abs(I - I_left))
print('Точность достигнута:', eps_left >= abs(I - I_left))
print('eps =', eps_S, ', I_S =', I_S, ', R = |I - I_S| =', abs(I - I_S))
print('Точность достигнута:', eps_S >= abs(I - I_S))

I_G = Gauss_3(FunctionPolynomial)
print('I_G =', I_G, ', R = |I - I_G| =', abs(I - I_G))

Шаг интегрирования для формулы левых прямоугольников: 4.098360655737705e-06
Число разбиений для формулы левых прямоугольников: 488000
Шаг интегрирования для формулы Симпсона: 0.3333333333333333
Число разбиений для формулы Симпсона: 6
I = -62.77333333333333
eps = 0.001 , I_left = -62.77302267781227 , R = |I - I_left| = 0.0003106555210621309
Точность достигнута: True
eps = 0.001 , I_S = -62.77407407407405 , R = |I - I_S| = 0.0007407407407171718
Точность достигнута: True
I_G = -62.773333333333326 , R = |I - I_G| = 7.105427357601002e-15


In [4]:
# Новый отрезок и функция
a_2 = 0
b_2 = 3
# Проверить разные точности
eps = 1e-12

def f(x):
    return 2.8 * math.sin(3**x)

# Точное значение интеграла и приближённые по формулам левых прямоугольников и Симпсона
I_2 = integr.quad(f, a_2, b_2)[0]
I_left2, I_left_prev2, h_left2, n_left2 = LeftRiemannSumByEps(f, a_2, b_2, eps_left)
I_S2, I_S_prev2, h_S2, n_S2 = SimpsonRuleByEps(f, a_2, b_2, eps)
print('I_2 =', I_2)
print()

print('Формула левых прямоугольников:')
print('h =', h_left2, 'n =', n_left2)
print('I_h =', I_left2, ', R_h = |I - I_h| =', abs(I_2 - I_left2))
print('Точность достигнута:', abs(I_2 - I_left2) < eps_left)
I_newleft2 = I_left2 + (I_left2 - I_left_prev2)/(2**1 - 1)
print('Уточнённое значение:', I_newleft2)
R_newleft2 = abs(I_2 - I_newleft2)
print('Уточнённая погрешность:', R_newleft2)
print('Точность достигнута:', R_newleft2 < eps_left)
print()

print('Формула Симпсона:')
print('h =', h_S2, 'n =', n_S2)
print('I_h =', I_S2, ', R_h = |I - I_h| =', abs(I_2 - I_S2))
print('Точность достигнута:', abs(I_2 - I_S2) < eps)
print('I_2h =', I_S_prev2, ', R_2h = |I - I_2h| =', abs(I_2 - I_S_prev2))
print('Точность достигнута:', abs(I_2 - I_S_prev2) < eps)
I_newS2 = I_S2 + (I_S2 - I_S_prev2)/(2**4 - 1)
print('Уточнённое значение:', I_newS2)
R_newS2 = abs(I_2 - I_newS2)
print('Уточнённая погрешность:', R_newS2)
print('Точность достигнута:', R_newS2 < eps)

I_2 = 1.6163730702821497

Формула левых прямоугольников:
h = 0.0029296875 n = 1024
I_h = 1.6158832356193642 , R_h = |I - I_h| = 0.0004898346627855155
Точность достигнута: True
Уточнённое значение: 1.6164101766397692
Уточнённая погрешность: 3.710635761944303e-05
Точность достигнута: True

Формула Симпсона:
h = 0.0003662109375 n = 8192
I_h = 1.6163730702822379 , R_h = |I - I_h| = 8.815170815523743e-14
Точность достигнута: True
I_2h = 1.6163730702835104 , R_2h = |I - I_2h| = 1.3606893389805919e-12
Точность достигнута: False
Уточнённое значение: 1.616373070282153
Уточнённая погрешность: 3.3306690738754696e-15
Точность достигнута: True
